In [1]:
import numpy as np 
import xarray as xr
import matplotlib.pyplot as plt
from pyresample.geometry import AreaDefinition
from pyresample.geometry import SwathDefinition
from pyresample import kd_tree

# Load data

In [2]:
path = "../../data/application/"


In [3]:
# Define original area
projection = "+proj=lcc +lat_1=35 +lat_2=65 +lat_0=52 +lon_0=10 +x_0=4000000 +y_0=2800000 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs"
width = 1200
height = 1100
area_extent = (3500000, 2100000, 4700000, 3200000)
area_def = AreaDefinition('original', 'Original grid', 'id', projection,width, height, area_extent)

# Define target grid
lons = np.linspace(6.38,7.48,30)
lats = np.linspace(50.27,50.97,30)
lon2d, lat2d = np.meshgrid(lons, lats)
target_grid = SwathDefinition(lons = lon2d, lats = lat2d)

In [4]:
# Functions for projection, interpolation and monthly maximum

def downsample(data, area_def, target_grid):
    result = kd_tree.resample_nearest(
        area_def, data, target_grid, radius_of_influence=50000, epsilon=0.5
    )
    return result


def xr_downsample(data, area_def, target_grid):
    result = xr.apply_ufunc(
        downsample,
        data,
        area_def,
        target_grid,
        input_core_dims=[["y", "x", "time"], [], []],
        output_core_dims=[["y", "x", "time"]],
        exclude_dims=set(("x", "y")),
        vectorize = True,
        dask = "allowed"
    )
    return result.max(dim = ["time"])


def transform_data(data, area_def, target_grid, year = ""):
    transformed_data = data.resample(time = "1MS").apply(xr_downsample, args = (area_def, target_grid))
    transformed_data = transformed_data.isel(time=(transformed_data.time.dt.month.isin([6,7,8])))
    xr.Dataset({"pr":transformed_data}).to_netcdf(f"../../data/application/{year}_month_max.nc")

In [5]:
# Training data
raw_data = xr.open_dataset(path + "1931_2020_raw.nc")
data = raw_data.pr.isel(time=(raw_data.time.dt.month.isin([6,7,8])))
transform_data(data, area_def, target_grid, year = "1931_2020")

# 2021
raw_data = xr.open_dataset(path + "2021_raw.nc")
data = raw_data.pr.isel(time=(raw_data.time.dt.month.isin([6,7,8])))
transform_data(data, area_def, target_grid, year = "2021")

# 2022
raw_data = xr.open_dataset(path + "2022_raw.nc")
data = raw_data.pr.isel(time=(raw_data.time.dt.month.isin([6,7,8])))
transform_data(data, area_def, target_grid, year = "2022")